## 数据集下载

In [1]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)
from sklearn.metrics import accuracy_score

os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

# IMDB数据集
ds = load_dataset("stanfordnlp/imdb")

## BERT测试

In [5]:
# 评价指标
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {"accuracy": accuracy_score(p.label_ids, preds)}

# BERT模型和tokenizer
model_name_bert = "google-bert/bert-base-uncased"
tokenizer_bert = AutoTokenizer.from_pretrained(model_name_bert)
model_bert = AutoModelForSequenceClassification.from_pretrained(model_name_bert, num_labels=2)


# 数据预处理
def preprocess_function_bert(examples):
    return tokenizer_bert(examples['text'], truncation=True)


encoded_ds_bert = ds.map(preprocess_function_bert, batched=True)

# 训练参数
training_args_bert = TrainingArguments(
    output_dir="./results_bert",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir="./logs_bert",
    logging_steps=10,
    load_best_model_at_end=True
)

# 训练
trainer_bert = Trainer(
    model=model_bert,
    args=training_args_bert,
    train_dataset=encoded_ds_bert["train"],
    eval_dataset=encoded_ds_bert["test"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer_bert,
    data_collator=DataCollatorWithPadding(tokenizer_bert)
)

trainer_bert.train()
results_bert = trainer_bert.evaluate()
print(f"BERT accuracy: {results_bert['eval_accuracy']:.4f}")

/root/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/root/miniconda3/lib/python3.12/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.246600,0.210196,0.919360
2,0.114800,0.246060,0.940560


BERT accuracy: 0.9194


## GPT-2 测试

In [6]:
# 加载GPT-2模型和tokenizer
model_name_gpt2 = "openai-community/gpt2"
tokenizer_gpt2 = AutoTokenizer.from_pretrained(model_name_gpt2)
model_gpt2 = AutoModelForSequenceClassification.from_pretrained(model_name_gpt2, num_labels=2)

# 指定填充token
tokenizer_gpt2.pad_token = tokenizer_gpt2.eos_token
model_gpt2.config.pad_token_id = tokenizer_gpt2.pad_token_id  # 手动设置pad_token_id

# 数据预处理
def preprocess_function_gpt2(examples):
    return tokenizer_gpt2(examples['text'], truncation=True)

encoded_ds_gpt2 = ds.map(preprocess_function_gpt2, batched=True)

# 训练参数
training_args_gpt2 = TrainingArguments(
    output_dir="./results_gpt2",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir="./logs_gpt2",
    logging_steps=10,
    load_best_model_at_end=True
)

trainer_gpt2 = Trainer(
    model=model_gpt2,
    args=training_args_gpt2,
    train_dataset=encoded_ds_gpt2["train"],
    eval_dataset=encoded_ds_gpt2["test"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer_gpt2,
    data_collator=DataCollatorWithPadding(tokenizer_gpt2)
)

# 训练并评估模型
trainer_gpt2.train()
results_gpt2 = trainer_gpt2.evaluate()
print(f"GPT-2 accuracy: {results_gpt2['eval_accuracy']:.4f}")


/root/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at openai-community/gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/root/miniconda3/lib/python3.12/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

## gemma-2-2b-it-bnb-4bit测试

In [3]:
import os
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

ds = load_dataset("stanfordnlp/imdb")
test_ds = ds['test'].select(range(200))

model_id = "unsloth/gemma-2-2b-it-bnb-4bit"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto')

# 计算准确率
def compute_accuracy(preds, labels):
    correct = 0
    for pred, label in zip(preds, labels):
        if ("positive" in pred.lower() and label == 1) or ("negative" in pred.lower() and label == 0):
            correct += 1
    return correct / len(labels)

# Zero-shot 情况下的 prompt
def zero_shot_prompt(text):
    return f"Classify the following movie review as positive or negative:\n\n{text}\n\nAnswer:"

# 2-shot 和 4-shot 的prompt
def fixed_n_shot_prompt(text, shots):
    if shots == 2:
        return (f"Classify the sentiment of the following reviews. Each review is labeled as either 'positive' or 'negative':\n"
                f"1. I love this movie! It was fantastic.\nLabel: positive\n"
                f"2. I didn't like this movie at all. It was boring.\nLabel: negative\n\n"
                f"Now classify the sentiment of this review:\n{text}\nSentiment:")
    elif shots == 4:
        return (f"Classify the sentiment of the following reviews. Each review is labeled as either 'positive' or 'negative':\n"
                f"1. I love this movie! It was fantastic.\nLabel: positive\n"
                f"2. I didn't like this movie at all. It was boring.\nLabel: negative\n"
                f"3. The plot was intriguing and the acting was superb.\nLabel: positive\n"
                f"4. The movie was a waste of time. Terrible.\nLabel: negative\n\n"
                f"Now classify the sentiment of this review:\n{text}\nSentiment:")

# 生成模型预测
def generate_predictions(model, tokenizer, dataset, shot_type="zero", shots=None):
    preds = []
    for example in dataset:
        text = example['text']
        if shot_type == "zero":
            prompt = zero_shot_prompt(text)
        elif shot_type == "n-shot":
            prompt = fixed_n_shot_prompt(text, shots)
        
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        with torch.no_grad():
            outputs = model.generate(**inputs, max_new_tokens=50)
        preds.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
    return preds

# Zero-shot 测试
zero_shot_preds = generate_predictions(model, tokenizer, test_ds, shot_type="zero")
zero_shot_accuracy = compute_accuracy(zero_shot_preds, [x['label'] for x in test_ds])
print(f"Zero-shot accuracy: {zero_shot_accuracy:.4f}")

# 2-shot 测试
two_shot_preds = generate_predictions(model, tokenizer, test_ds, shot_type="n-shot", shots=2)
two_shot_accuracy = compute_accuracy(two_shot_preds, [x['label'] for x in test_ds])
print(f"2-shot accuracy: {two_shot_accuracy:.4f}")

# 4-shot 测试
four_shot_preds = generate_predictions(model, tokenizer, test_ds, shot_type="n-shot", shots=4)
four_shot_accuracy = compute_accuracy(four_shot_preds, [x['label'] for x in test_ds])
print(f"4-shot accuracy: {four_shot_accuracy:.4f}")


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Zero-shot accuracy: 1.0000
2-shot accuracy: 1.0000
4-shot accuracy: 1.0000


## glm4-flash测试

In [5]:
import random
from datasets import load_dataset
from sklearn.metrics import accuracy_score
from zhipuai import ZhipuAI

# 加载数据集
dataset = load_dataset("stanfordnlp/imdb")
test_data = dataset['test'].select(range(200))

client = ZhipuAI(api_key="c6c970d4cd4f13745d08adf923aefb7d.P7EC5rOHL54sJwaB")

# 生成 zero-shot prompt
def zero_shot_prompt(text):
    return f"Classify the sentiment of the following movie review as positive or negative:\n\n{text}\n\nAnswer:"

# 生成 n-shot prompt
def fixed_n_shot_prompt(text, shots):
    if shots == 2:
        return (f"Classify the sentiment of the following reviews. Each review is labeled as either 'positive' or 'negative':\n"
                f"1. I love this movie! It was fantastic.\nLabel: positive\n"
                f"2. I didn't like this movie at all. It was boring.\nLabel: negative\n\n"
                f"Now classify the sentiment of this review:\n{text}\nSentiment:")
    elif shots == 4:
        return (f"Classify the sentiment of the following reviews. Each review is labeled as either 'positive' or 'negative':\n"
                f"1. I love this movie! It was fantastic.\nLabel: positive\n"
                f"2. I didn't like this movie at all. It was boring.\nLabel: negative\n"
                f"3. The plot was intriguing and the acting was superb.\nLabel: positive\n"
                f"4. The movie was a waste of time. Terrible.\nLabel: negative\n\n"
                f"Now classify the sentiment of this review:\n{text}\nSentiment:")

# 获取模型预测
def get_prediction(prompt):
    response = client.chat.completions.create(
        model="glm-4-flash", 
        messages=[{"role": "user", "content": prompt}],
    )
    
    choices = response.choices
    if choices and choices[0].message:
        return choices[0].message.content.strip()
    else:
        raise ValueError("Unexpected response format: 'message' not found in response.")

# 计算准确率
def compute_accuracy(preds, labels):
    correct = 0
    for pred, label in zip(preds, labels):
        if ("positive" in pred.lower() and label == 1) or ("negative" in pred.lower() and label == 0):
            correct += 1
    return correct / len(labels)

# 生成模型预测
def generate_predictions(test_data, shot_type="zero", shots=None):
    preds = []
    for example in test_data:
        review_text = example['text']
        if shot_type == "zero":
            prompt = zero_shot_prompt(review_text)
        elif shot_type == "n-shot":
            prompt = fixed_n_shot_prompt(review_text, shots)
        
        pred = get_prediction(prompt)
        preds.append(pred)
    return preds

# Zero-shot 测试
zero_shot_preds = generate_predictions(test_data, shot_type="zero")
zero_shot_accuracy = compute_accuracy(zero_shot_preds, [x['label'] for x in test_data])
print(f"Zero-shot accuracy: {zero_shot_accuracy:.4f}")

# 2-shot 测试
two_shot_preds = generate_predictions(test_data, shot_type="n-shot", shots=2)
two_shot_accuracy = compute_accuracy(two_shot_preds, [x['label'] for x in test_data])
print(f"2-shot accuracy: {two_shot_accuracy:.4f}")

# 4-shot 测试
four_shot_preds = generate_predictions(test_data, shot_type="n-shot", shots=4)
four_shot_accuracy = compute_accuracy(four_shot_preds, [x['label'] for x in test_data])
print(f"4-shot accuracy: {four_shot_accuracy:.4f}")


Zero-shot accuracy: 0.9600
2-shot accuracy: 0.9650
4-shot accuracy: 0.9650
